In [1]:
import os
from scipy import misc

file_dirs = []
for root, dirs, files in os.walk("/home/vietdoan/Downloads/Parking lots/PKLot/PKLotSegmented/UFPR04", topdown=False):
    for name in files:
        file_dirs.append(os.path.join(root, name))

In [2]:
import random
random.shuffle(file_dirs)
num_samples = 30000
X = []
Y = []
for i in range(0, num_samples):
    f = misc.imread(file_dirs[i])
    f = misc.imresize(f, (48, 64))
    X.append(f)
    folders = file_dirs[i].split('/')
    label = 1 if folders[len(folders) - 2] == 'Occupied' else 0
    Y.append(label)


In [3]:
import numpy as np
X = np.array(X)
Y = np.array(Y)

In [4]:
(x_train, y_train) = (X[:num_samples - 1000], Y[:num_samples - 1000])
(x_test, y_test) = (X[num_samples - 1000:num_samples], Y[num_samples - 1000:num_samples])

In [5]:
batch_size = 128
num_classes = 2
epochs = 11
img_rows, img_cols = 48, 64

In [6]:
import keras
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [8]:
if keras.backend.image_data_format() == 'channels_first':
    input_shape = (3, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 3)

In [9]:
input_shape

(48, 64, 3)

In [10]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv2D(10, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
model.add(Conv2D(20, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
model.add(Conv2D(30, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [13]:
model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 44, 60, 10)        760       
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 60, 10)        40        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 30, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 26, 20)        5020      
_________________________________________________________________
batch_normalization_2 (Batch (None, 18, 26, 20)        80        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 13, 20)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 9, 30)          15030     
__________

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 29000 samples, validate on 1000 samples
Epoch 1/11
29000/29000 [==============================] - 239s - loss: 0.0616 - acc: 0.9806 - val_loss: 1.7507 - val_acc: 0.5510
Epoch 2/11
29000/29000 [==============================] - 236s - loss: 0.0098 - acc: 0.9971 - val_loss: 0.3742 - val_acc: 0.8410
Epoch 3/11
29000/29000 [==============================] - 240s - loss: 0.0052 - acc: 0.9984 - val_loss: 0.0237 - val_acc: 0.9930
Epoch 4/11
29000/29000 [==============================] - 232s - loss: 0.0034 - acc: 0.9990 - val_loss: 0.0054 - val_acc: 0.9980
Epoch 5/11
29000/29000 [==============================] - 239s - loss: 0.0022 - acc: 0.9994 - val_loss: 0.0075 - val_acc: 0.9970
Epoch 6/11
29000/29000 [==============================] - 233s - loss: 0.0014 - acc: 0.9997 - val_loss: 0.0018 - val_acc: 0.9990
Epoch 7/11
29000/29000 [==============================] - 232s - loss: 0.0012 - acc: 0.9998 - val_loss: 0.0147 - val_acc: 0.9930
Epoch 8/11
29000/29000 [========================

In [17]:
x_val = []
y_val = []
for i in range(num_samples, num_samples + 10000):
    f = misc.imread(file_dirs[i])
    f = misc.imresize(f, (48, 64))
    x_val.append(f)
    folders = file_dirs[i].split('/')
    label = 1 if folders[len(folders) - 2] == 'Occupied' else 0
    y_val.append(label)

In [18]:
x_val = np.array(x_val)
y_val = np.array(y_val)

In [28]:
x_val = x_val.astype('float32')
x_val /= 255
prediction = model.predict(x_val)

In [20]:
y_val = keras.utils.to_categorical(y_val, num_classes)

In [34]:
acc_val = np.mean(np.round(prediction) == y_val)

In [35]:
acc_val

0.43359999999999999

In [36]:
np.round(prediction)

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)

In [37]:
x_val

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        ..., 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
     